<a href="https://colab.research.google.com/github/R12942159/NTU_ML/blob/Hw5/hand_craft_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.optim import SGD, Adam, AdamW
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.datasets import make_classification
seed = 3047
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [ ]:
!gdown 1o0m3jyfmetUOJ146TqHuEGUWwQyC7JXV
!gdown 1B5OC3R0yM8F7yjoYOKu3t08QZalcr7DC
!gdown 1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2

In [156]:
class SVM(nn.Module):
    def __init__(self):
        super(SVM, self).__init__()

        self.w = nn.Parameter(torch.randn((4, 1)).to(torch.float32))
        self._gamma = torch.nn.Parameter(torch.FloatTensor([1]),
                                         requires_grad = True)

        #define kernal & feature selection
        self.f = nn.Sequential(
                  nn.Linear(107, 4),
                  )

    def kernel(self, x):
        x = self.f(x)
        return x

    def forward(self, x):
        f = torch.matmul(self.kernel(x), self.w)
        return f

In [157]:
class HingeLoss(nn.Module):
  def __init__(self, C):
    super(HingeLoss, self).__init__()
    self.C = C

  def forward(self, y, f):
    loss = torch.sum(self.C * nn.functional.relu(1 - y * f)) / y.shape[0]
    return loss

In [158]:
class TrainDataset(Dataset):
  def __init__(self,split, mu=None, std=None):

    X = pd.read_csv(f"{split}.csv")


    X_c = X[['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']].values
    X_d = X.drop(['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss', 'y'], 1).values
    Y = X['y'].values.reshape(-1) * 2 - 1   #1 or -1

    X_c, mu, std = self.normalize(X_c, mu, std)

    X = np.concatenate((X_d, X_c, np.ones((X.shape[0], 1))), 1)
    self.Y = torch.from_numpy(Y).to(torch.float32)
    self.X = torch.from_numpy(X).to(torch.float32)
    self.mu = mu
    self.std = std

  def normalize(self, X, mu=None, std=None):
    #ToDo
    mu = np.mean(X.astype('float64') , axis = 0)
    std = np.std(X.astype('float64') , axis = 0)
    mu[3:5] = ([1e-10,1e-10])
    std[3:5] = ([1,1])
    X = (X - mu) / (std + 1e-10)

    return X, mu, std

  def __len__(self):
    return self.X.size(0)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [159]:
class TestDataset(Dataset):
  def __init__(self, mu, std):
    X = pd.read_csv("X_test")
    X_c = X[['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']].values
    X_d = X.drop(['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss'], 1).values
    X_c= self.normalize(X_c, mu, std)
    X = np.concatenate((X_d, X_c, np.ones((X.shape[0], 1))), 1)
    self.X = torch.from_numpy(X).to(torch.float32)

  def normalize(self, X, mu_x, std_x):
    X = (X - mu_x) / (std_x + 1e-10)

    return X

  def __len__(self):
    return self.X.size(0)

  def __getitem__(self, idx):
    return self.X[idx]

In [160]:
def train(train_data, val_data, model, optim, C, device='cuda:0'):
    epoch = 100
    objective = HingeLoss(C)
    steps = 0
    best = 0

    for e in range(epoch):
      for tr in train_data:
        steps += 1
        x_train, y_train = tr
        x_train, y_train = x_train.to(device), y_train.to(device)

        # TODO : try to implement gradient descent
        pred = model(x_train)
        loss = objective(pred, y_train)

        optim.zero_grad()
        loss.backward()
        optim.step()


        if steps % 100 == 0:
          model.eval()
          with torch.no_grad():
            acc = []
            for val in val_data:
              x_val, y_val = val
              x_val , y_val = x_val.to(device), y_val.to(device)
              pred = model(x_val).squeeze(1)
              pred = (pred > 0) * 2 - 1

              result = (y_val == pred)
              acc += [(float(result.sum()) / result.size(0))]
            acc = sum(acc) / len(acc)
            print(f'Steps {steps}| Train Loss = {loss.item()}| Val acc = {acc}')

            if acc > best:
              torch.save(model.state_dict(), 'best.ckpt')
              print("**************************************")
              best = acc
          model.train()
    return model

In [161]:
lr = 0.0001
batch = 2048
C = 100
device = 'cuda:0'

In [ ]:
trainset = TrainDataset('train')
devset = TrainDataset('val', trainset.mu, trainset.std)
testset = TestDataset(trainset.mu, trainset.std)

train_dataloader = DataLoader(trainset, batch, True, drop_last=False)
val_dataloader = DataLoader(devset, 1, False)
test_dataloader = DataLoader(testset, 1, False)

model = SVM().to(device)
optim = Adam(model.parameters(), lr)
model = train(train_dataloader, val_dataloader, model, optim, C, device)

In [163]:
best_model = model
best_model.load_state_dict(torch.load('best.ckpt'))
best_model = best_model.eval()
# TODO: predict x_test
y_test = []
for x in test_dataloader:
  x = x.to(device)
  y = best_model(x)
  y_test.append(((y > 0) * 1).item())



In [164]:
import csv
with open('predict.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['id','label'])
    for i in range(len(y_test)):
      writer.writerow( [i + 1, int(y_test[i])] )